In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv("../data/foodb/Content.csv")

C:\Users\oszud\AppData\Local\Temp\ipykernel_17024\2274376406.py:1: DtypeWarning: Columns (4,5,6,7,8,9,13,14,21,22,24) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("../data/foodb/Content.csv")


Original unit of protein mg/100g

In [ ]:
data_prot = data[data.orig_source_name == "Protein"]
data_prot.head(3)

,id,source_id,source_type,food_id,orig_food_id,orig_food_common_name,orig_food_scientific_name,orig_food_part,orig_source_id,orig_source_name,...,citation_type,creator_id,updater_id,created_at,updated_at,orig_method,orig_unit_expression,standard_content,preparation_type,export
399,400,2,Nutrient,4,9148,"Kiwifruit, green, raw",NaN,NaN,203,Protein,...,DATABASE,NaN,NaN,2014-11-05 13:42:17 UTC,2019-12-11 20:23:59 UTC,NaN,NaN,1140.0,raw,0
400,401,2,Nutrient,4,9445,"Kiwifruit, gold, raw",NaN,NaN,203,Protein,...,DATABASE,NaN,NaN,2014-11-05 13:42:17 UTC,2019-12-11 20:23:59 UTC,NaN,NaN,1230.0,raw,0
410,411,2,Nutrient,5,2026,"Spices, onion powder",NaN,NaN,203,Protein,...,DATABASE,NaN,NaN,2014-11-05 13:42:17 UTC,2019-12-11 20:23:59 UTC,NaN,NaN,10410.0,dried or powder,0


As you see - there is also "Protein, total-N" type of records. Those records tell us about nitrogen content. Internet says that sometimes it is used to estimate how much protein is actually in a food. However, because it is not that straightforward - I suggest we do not dig into it right now. If we won't obtain enough info - it might be somehow useful.

Based on 
```
 from ydata_profiling import ProfileReport

 profile = ProfileReport(data_prot, title='Protein FooDB Report')
 profile
```
and
```
 data_prot.COLNAME.unique()
```
we'll drop some columns

In [ ]:
data_prot = data[data.orig_source_name == "Protein"]
data_prot = data_prot.drop(columns=['creator_id', 'updater_id', 'created_at', 'updated_at', 
                                    'orig_citation', 'citation', 'citation_type',
                                    'orig_unit', 'source_id', 'orig_source_name', 'orig_source_id',
                                    'orig_food_scientific_name', 'orig_food_part', 'orig_min', 'orig_max',
                                    'orig_method', 'orig_unit_expression', 'source_type'])

In [ ]:
data_prot.head(3)

,id,food_id,orig_food_id,orig_food_common_name,orig_content,standard_content,preparation_type,export
399,400,4,9148,"Kiwifruit, green, raw",1140.0,1140.0,raw,0
400,401,4,9445,"Kiwifruit, gold, raw",1230.0,1230.0,raw,0
410,411,5,2026,"Spices, onion powder",10410.0,10410.0,dried or powder,0


In [ ]:
data_prot[data_prot['orig_food_common_name'].str.contains('Onions')][:5]

,id,food_id,orig_food_id,orig_food_common_name,orig_content,standard_content,preparation_type,export
411,412,5,11282,"Onions, raw",1100.0,1100.0,raw,0
412,413,5,11283,"Onions, cooked, boiled, drained, without salt",1360.0,1360.0,cooked,0
413,414,5,11284,"Onions, dehydrated flakes",8950.0,8950.0,dried or powder,0
414,415,5,11285,"Onions, canned, solids and liquids",850.0,850.0,NaN,0
415,416,5,11286,"Onions, yellow, sauteed",950.0,950.0,cooked,0


In [ ]:
names = data.orig_source_name.unique()
prot_names = []
for name in names:
    n = str(name)
    if "Protein" in n:
        prot_names.append(name)
        
prot_names.remove('Protein, total-N')
prot_names

['Protein, total', 'Protein', 'Adjusted Protein']

In [ ]:
foodb_ingr = set()

for prot_name in prot_names:
    data_prot = data[data.orig_source_name == prot_name]
    data_prot = data_prot.drop(columns=['creator_id', 'updater_id', 'created_at', 'updated_at', 
                                    'orig_citation', 'citation', 'citation_type',
                                    'orig_unit', 'source_id', 'orig_source_name', 'orig_source_id',
                                    'orig_food_scientific_name', 'orig_food_part', 'orig_min', 'orig_max',
                                    'orig_method', 'orig_unit_expression', 'source_type'])
    
    for record in data_prot['orig_food_common_name']:
        items = record.split(',')
        first_item = items[0].strip()
        foodb_ingr.add(first_item)

In [ ]:
len(foodb_ingr)

1331